In [50]:
import keras
import tensorflow as tf
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50  import ResNet50, preprocess_input
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout

In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
# Пути к наборам данных
train_dir = '/content/drive/MyDrive/datalab3/train'
test_dir = '/content/drive/MyDrive/datalab3/val' 
# Размер изображений
img_width, img_height = 300, 300
# Размер мини-выборки
batch_size = 80
# Кол-во изображений для обучения
nb_train_samples = 1027
# Кол-во изображений для теста
nb_test_samples = 256

In [53]:
# Создание генератора изображений
datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [54]:
# Создание экземпляра модели сети ResNet50
resnet = ResNet50(
    input_shape=(img_width, img_height, 3),
    include_top=False,
    weights="imagenet"
)

resnet.trainable = False

resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 300, 300, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 306, 306, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 150, 150, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 150, 150, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [55]:
# Создание модели составной сети (Составная сеть)
model = Sequential()
# Добавляем сверточные слои
model.add(resnet)
# Преобразуем двумерный массив MobileNet в одномерный
model.add(Flatten())
# Полносвязный слой
model.add(Dense(256, activation='relu'))
# Слой регуляризации (для предотвращения переобучения)
model.add(Dropout(0.5))
# Кол-во классов
model.add(Dense(1, activation='sigmoid'))
model.build(input_shape=(None, img_width, img_height, 3))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
module_wrapper_5 (ModuleWrap (None, 10, 10, 2048)      23587712  
_________________________________________________________________
flatten_5 (Flatten)          (None, 204800)            0         
_________________________________________________________________
dense_10 (Dense)             (None, 256)               52429056  
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 257       
Total params: 76,017,025
Trainable params: 52,429,313
Non-trainable params: 23,587,712
_________________________________________________________________


In [56]:
model.compile(loss='binary_crossentropy',
              optimizer='SGD', 
              metrics=['accuracy'])

In [57]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=7,
    validation_data=test_generator,
    validation_steps=nb_test_samples // batch_size)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/7
12/12 [==============================] - 13s 810ms/step - loss: 22.6246 - accuracy: 0.5115 - val_loss: 0.6947 - val_accuracy: 0.5083
Epoch 2/7
12/12 [==============================] - 8s 730ms/step - loss: 0.6951 - accuracy: 0.5131 - val_loss: 0.6861 - val_accuracy: 0.7292
Epoch 3/7
12/12 [==============================] - 8s 726ms/step - loss: 0.6917 - accuracy: 0.5364 - val_loss: 0.6871 - val_accuracy: 0.7375
Epoch 4/7
12/12 [==============================] - 8s 719ms/step - loss: 0.6926 - accuracy: 0.5124 - val_loss: 0.6840 - val_accuracy: 0.5125
Epoch 5/7
12/12 [==============================] - 8s 718ms/step - loss: 0.6917 - accuracy: 0.5321 - val_loss: 0.6871 - val_accuracy: 0.7375
Epoch 6/7
12/12 [==============================] - 8s 716ms/step - loss: 0.6931 - accuracy: 0.5061 - val_loss: 0.6871 - val_accuracy: 0.7292
Epoch 7/7
12/12 [==============================] - 8s 716ms/step - loss: 0.6932 - accuracy: 0.4908 - val_loss: 0.6868 - val_accuracy: 0.7500


In [58]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print('Точность работы на тестовых данных: %.2f%%' % (scores[1]*100))

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1948: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


Точность работы на тестовых данных: 73.75%
